In [1]:
from datasets import load_dataset, Dataset, concatenate_datasets

dataset = load_dataset("blinoff/kinopoisk")
dataset = dataset["train"]
dataset

Found cached dataset kinopoisk (/home/kosenko/.cache/huggingface/datasets/blinoff___kinopoisk/simple/1.0.0/62f52027aea59f64f49c7b16165b82cb4dc45031bad3660c2719bf2a6ea4a44e)


  0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['content', 'title', 'grade3', 'movie_name', 'part', 'review_id', 'author', 'date', 'grade10', 'Idx'],
    num_rows: 36591
})

In [9]:
print(dataset[10]["content"])


«Блеф» — комедия Серджио Корбуччи 1976 года. Один из любимых фильмов зрителей в советский период.

В данной ленте идет речь о мошеннике Феликсе (Феличе — что означает счастье) и о его многочисленных аферах. Как-то раз Феликс сбегает из тюрьмы, не дав сделать то же самое другому заключенному, после чего ему помогают два незнакомых ему человека, что удивляет самого героя. Но как оказывается впоследствии, его приняли не за того. Теперь он должен освободить Филиппа Бэнга. Но на этом приключения не заканчивается.

Освобождённый заключенный предлагает сотрудничество (после нескольких проверок). Филипп (а именно им оказался заключенный) хочет провернуть дело, чтобы отомстить своей давней знакомой Бель Дюк.

Параллельно блефу главных героев развивается любовная линия Феликса и дочери Филиппа — Шарлотты.

Фильм очень легкий, веселый, с интересными поворотами сюжета (что является явным плюсом). Андриано Челентано бесподобен как всегда, очень интересно смотреть на его героя, наблюдать за его вых

In [12]:
# dataset.push_to_hub("dim/kinopoisk_raw", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/37 [00:00<?, ?ba/s]

### convert to prompts


In [3]:
dataset[0]

{'content': '\n"Блеф»\xa0—\xa0одна из\xa0моих самых любимых комедий.\n\nЭтот фильм я\xa0наверно смотрел раз\xa0сто, нет\xa0я конечно блефую, я\xa0видел его\xa0куда больше. Не\xa0могу не\xa0выразить своё восхищение главными действующими лицами этого фильма. Начну с\xa0Адриано Челентано для\xa0которого как\xa0я считаю это\xa0лучшая роль в\xa0кино. Великолепный актёр, неплохой певец, странно что\xa0на его\xa0родине в\xa0Италии его\xa0песни мало кто\xa0слушает. Ну\xa0я думаю что\xa0и итальянцы и\xa0французы привыкли к\xa0тому, что\xa0у нас\xa0до сих\xa0их актёры популярней чем\xa0даже на\xa0своей родине. Да, такой вот\xa0парадокс. Челентано конечно профессионал своего дела, комик с\xa0серьёзным выражением лица. Он\xa0смешон ещё\xa0и потому, что\xa0одновременно так\xa0серъёзен. Адриано браво!\n\nА\xa0теперь несколько слов об\xa0Энтони Куине. Да\xa0тот самый горбун из\xa0Нотр-дама. Собор Парижской Богоматери, оригинальная версия, кто\xa0не смотрел рекомендую. С\xa0ним как-то приключилась одн

In [4]:
set(dataset["grade3"])

{'Bad', 'Good', 'Neutral'}

In [10]:
prompts = []
for item in dataset:
    grade = item["grade3"]
    movie_name = item["movie_name"]
    content = item["content"]
    grade_map = {
        "Bad":"плохой",
        "Good":"хороший",
        "Neutral":"нейтральный",
    }
    ru_grade = grade_map[grade]
    prompt = f"напиши {ru_grade} отзыв на фильм {movie_name}"
    # print(prompt)
    # print(content)
    # print("="*100)
    # print("="*100)
    # print("="*100)
    prompts.append({
        "prompt": prompt,
        "content": content
    })

In [11]:
len(prompts)

36591

In [12]:
dataset = Dataset.from_list(prompts)

In [13]:
dataset.push_to_hub("dim/kinopoisk_prompts", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/37 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Updating downloaded metadata with the new split.


In [14]:
dataset

Dataset({
    features: ['prompt', 'content'],
    num_rows: 36591
})

In [3]:
from datasets import load_dataset, Dataset, concatenate_datasets

dataset = load_dataset("dim/kinopoisk_prompts")
dataset = dataset["train"]
dataset

Found cached dataset parquet (/home/kosenko/.cache/huggingface/datasets/dim___parquet/dim--kinopoisk_prompts-9641473c3c5807e6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['prompt', 'content'],
    num_rows: 36591
})

In [4]:
dataset[0]

{'prompt': 'напиши хороший отзыв на фильм Блеф (1976)',
 'content': '\n"Блеф»\xa0—\xa0одна из\xa0моих самых любимых комедий.\n\nЭтот фильм я\xa0наверно смотрел раз\xa0сто, нет\xa0я конечно блефую, я\xa0видел его\xa0куда больше. Не\xa0могу не\xa0выразить своё восхищение главными действующими лицами этого фильма. Начну с\xa0Адриано Челентано для\xa0которого как\xa0я считаю это\xa0лучшая роль в\xa0кино. Великолепный актёр, неплохой певец, странно что\xa0на его\xa0родине в\xa0Италии его\xa0песни мало кто\xa0слушает. Ну\xa0я думаю что\xa0и итальянцы и\xa0французы привыкли к\xa0тому, что\xa0у нас\xa0до сих\xa0их актёры популярней чем\xa0даже на\xa0своей родине. Да, такой вот\xa0парадокс. Челентано конечно профессионал своего дела, комик с\xa0серьёзным выражением лица. Он\xa0смешон ещё\xa0и потому, что\xa0одновременно так\xa0серъёзен. Адриано браво!\n\nА\xa0теперь несколько слов об\xa0Энтони Куине. Да\xa0тот самый горбун из\xa0Нотр-дама. Собор Парижской Богоматери, оригинальная версия, кто\xa